# String arithmetic

Imagine you were to implement `math.pow` from scratch, would you be able to do it? And, if you were not allowed to use the muliplication function? And not the addition function either? In fact you were not allowed to use any function that returns a number, and not allowed to use numbers altogether?

It shouldn't be too difficult. In fact, when we do arithmetic with pencil and paper, we represent numbers as strings. We just need to translate those pencil and paper operations to code.

This is an exercise I did to familiarize myself with writing classes in Python and the so called "dunder" methods in particular.

Let's start with some static variables and the initiator.

In [ ]:
class StrNum(object):
    
    # Defining 32-bit numbers in little-endian
    digit_to_binary = {
        '0': '00000000000000000000000000000000',
        '1': '10000000000000000000000000000000',
        '2': '01000000000000000000000000000000',
        '3': '11000000000000000000000000000000',
        '4': '00100000000000000000000000000000',
        '5': '10100000000000000000000000000000',
        '6': '01100000000000000000000000000000',
        '7': '11100000000000000000000000000000',
        '8': '00010000000000000000000000000000',
        '9': '10010000000000000000000000000000'}

    binary_to_digit = {b: d for d, b in digit_to_binary.items()}
    
    def __init__(self, s, binary=False):
        # should have validated s, but trusting the client instead
        if binary:
            self._value = s
        elif s in StrNum.digit_to_binary:
            self._value = StrNum.digit_to_binary[s]
        else:
            n = zero
            sgn = one
            for d in s:
                if d == '-':
                    sgn = -sgn
                else:
                    n *= ten
                    n += StrNum(d)
            self._value = (sgn * n)._value
    

Numbers are represented  by 32 bits in little-endian format, which means that the least significant bit is the first bit. To represent negative numbers, we'll use *two's complement* format.

The initiator takes a string that represents a number in decimal or binary. If the string represents a number in binary, the string must be 32 bits long. If the string represents a number in decimal, the string may start with a '-' character to represent a negative number.

In this code, `zero`, `one`, `ten` (equal to `StrNum('0')`, `StrNum('1')`, and `StrNum('10')` respectively), and the other `StrNum` operations used in the initiator are defined later.

In [ ]:
    def __add__(self, other):

        def sum_bits():
            carry = '0'
            for x in zip(self._value, other._value):
                if x == ('0', '0'):
                    yield carry
                    carry = '0'
                elif x == ('1', '1'):
                    yield carry
                    carry = '1'
                else:
                    yield '1' if carry == '0' else '0'

        return StrNum(''.join(sum_bits()), binary=True)

Addition (`+`) is defined through the "dunder method" (a.k.a. "magic method") `__add__`. This method is called indirectly by `__init__()` defined above. It allows for expressions such as `a + b`, where `a` and `b` are `StrNum` instances. If `a` is a variable, it can also be used in expressions such as `a += b`. It is used that way not only in the initiator, but throughout the code that defines the `StrNum` class.

In [ ]:
    def __lshift__(self, other):
        b = one
        bits = deque(self._value)
        while b <= other:
            bits.appendleft('0')
            bits.pop()
            b = b + one
        return StrNum(''.join(bits), binary=True)

    def __rshift__(self, other):
        b = one
        bits = deque(self._value)
        sign_bit = '1' if self.__is_negative() else '0'
        while b <= other:
            bits.append(sign_bit)
            bits.popleft()
            b += one
        return StrNum(''.join(bits), binary=True)

Left shift (`<<`) and right shift (`>>`) are defined by the dunder methods `__lshift__` and `__rshift__`. Because of the little endian representation, a left shift shifts the bits to the right and vice versa for right shift. For right shifts, we fill in with the sign bit.

In [ ]:
    def __neg__(self):
        one_complement = ''.join('1' if c == '0' else '0' for c in self._value)
        return StrNum(one_complement, binary=True) + one

To negate a number when using two's complement representation, we invert the bits and add 1.

In [ ]:
    def __sub__(self, other):
        return self + -other

Subtraction is simply adding the opposite (defined above by `__neg__`).

In [ ]:
    def __is_negative(self):
        return ('1', '1') in zip(self._value, '00000000000000000000000000000001')

Whether a number is negative or positive is determined by the sign bit (the rightmost bit in little-endian representation). Since we cannot use an index to reference that bit (indices are numbers and numbers are not allowed), we resort to a little trick.

In [ ]:
    def __abs__(self):
        return -self if self.__is_negative() else self

    def __eq__(self, other):
        return self._value == other._value

    def __ne__(self, other):
        return self._value != other._value

    def __gt__(self, other):
        return (other - self).__is_negative()

    def __ge__(self, other):
        return self == other or self > other

    def __lt__(self, other):
        return (self - other).__is_negative()

    def __le__(self, other):
        return self == other or self < other

The above definitions should be straight-forward.

The definitions of `__mult__`, `__divmod__`, and `__pow__` are based on the "number climber" algorithm: Given a number `n`, generate a sequence of numbers starting at 1 and ending at `n` such that each number in the sequence except the first is the double or the double plus one of the preceding number in the sequence. For example, given `102`, the sequence is `1, 3, 6, 12, 25, 51, 102`, because `3 == 2 * 1 + 1`, `6 == 2 * 3`, `12 == 2 * 6`, `25 == 2 * 12 + 1`, `51 == 2 * 25 + 1`, and `102 == 2 * 51`. Notice that this sequence is unique.

The number climber can be illustrated by the following table:

| A   |  B  |  C  |
|-----|-----|-----|
| 102 | 128 |   0 |
|  38 |  64 |   1 |
|   6 |  32 |   3 |
|   6 |  16 |   6 |
|   6 |   8 |  12 |
|   2 |   4 |  25 |
|   0 |   2 |  51 |
|   0 |   1 | 102 |

Let's refer to the given number as $n$, the numbers in one row of the table as $a$, $b$, and $c$, and the numbers in the following row as $a'$, $b'$, and $c'$. The table is initialized with $a = n$, $b$ = a power of 2 greater than $n$, and $c = 0$. To go from one row to the next:

- $b' = b / 2$
- if $b' \le a$, then $a' = a - b'$ 
- if $b' \le a$, then $c' = 2c + 1$, else $c' = 2c$

The last row is reached when $b = 1$.

For each row, we have:
1. $0 \le a < b$
2. $a + bc = n$

So, when $b = 1$, $0 \le a < 1$, which implies $a = 0$ and $c = n$. We also have that each number in column C is the double or double plus 1 of the number in the preceding row, and, since it ends in $n$, this column contains the number climber sequence.

The number climber can be used to "climb" to a product of $nk$ for some number $k$. Imagine a fourth column containg the product $ck$. When $c' = 2c$, $c'k = 2ck$, and when $c' = 2c + 1$, $c'k = 2ck + k$. For example, if $k = 12$:

| A   |  B  |  C  |  D  |
|-----|-----|-----|-----|
| 102 | 128 |   0 |   0 |
|  38 |  64 |   1 |  12 |
|   6 |  32 |   3 |  36 |
|   6 |  16 |   6 |  72 |
|   6 |   8 |  12 | 144 |
|   2 |   4 |  25 | 300 |
|   0 |   2 |  51 | 612 |
|   0 |   1 | 102 |1224 |

Note that we don't really need columns B and C. In column A, we could have the bits of $n$ from most significant bit to least significant bit. The number $c$ doubles for each 0 bit in $n$ and doubles plus one for each 1 bit in $n$. Note that 102 is 1100110 in binary

|A  | 1 | 1 | 0 |  0 |  1|  1 |   0|
|---|---|---|---|----|---|----|----|
|D  |12 |36 |72 |144 |300|612 |1224|

This explains the `__mul__` method below. The `__divmod__` and `__pow__` methods are similar adaptations of the number climber. 

In [ ]:
    def __mul__(self, other):
        if self.__is_negative():
            return -(-self * other)
        result = zero
        for bit in reversed(self._value):
            result <<= one
            if bit == '1':
                result += other
        return result

Note that the code above is simpler than what we have implemented: `__mul__(self, other, mod=None)`, which returns `self * other % mod` if `mod` is not `None`. 

In [ ]:
    def __divmod__(self, other):
        # neither self nor other is negative
        q, r = zero, zero
        # dividend = the number we're climbing
        # divisor = other
        # dividend = q * divisor + r, 0 <= r < divisor
        for b in reversed(self._value): 
            q, r = q << one, r << one
            if b == '1':
                r += one
            if r >= other:
                r -= other
                q += one
        return q, r

The definition of `__divmod__` above applies only when both arguments are positive. More care must be taken to cover cases where one or both are negative, the gist of it is captured here.

In [ ]:
    def __pow__(self, exponent, mod=None):
        base = self % mod if mod else self
        power = one
        for bit in reversed(exponent._value):
            power = power.__mul__(power, mod)
            if bit == '1':
                power = power.__mul__(base, mod)
        return power

The above method is called when the function `pow()` is called on three StrNum objects. `self` and `mod` may be negative, but `exponent` must be non-negative and `mod` must be non-zero.

In [ ]:
    def __str__(self):
        if self == zero: return '0'
        if self < zero: return '-' + str(-self)
        result = ""
        s = self
        while s > zero:
            s, r = s.__divmod__(ten)
            result = StrNum.binary_to_digit[r._value] + result
        return result

The string representation of a `StrNum` object is the decimal string representation. For example, the string representation of `StrNum('123')` is `'123'`.

In [ ]:
    def __repr__(self):
        return f"StrNum('{self}')"

The `repr` of a `StrNum` object is an expression that evaluates to the object or to one that is equal. For example, the `repr` of `StrNum('123')` is `"StrNum('123')"`.

In [ ]:
    def __hash__(self):
        return hash(self._value)

Note that `StrNum` objects are immutable and hashable. The hash of a `StrNum` object is the hash of its `_value` property.

In [ ]:
zero = StrNum('0')
one  = StrNum('1')
ten  = StrNum('01010000000000000000000000000000', binary=True)

Finally, the constants `zero`, `one` and `ten` are defined outside the class definition. 

## Verification

In [ ]:
from src.strmath import *
from random import randrange

In [ ]:
bound = int(1e9)
base = str(randrange(-bound, bound + 1))
exponent = str(randrange(0, bound + 1))
x = randrange(-bound, bound)
modulus = str(bound) if x == 0 else str(x)

In [ ]:
base, exponent, modulus

In [ ]:
pow(int(base), int(exponent), int(modulus))

In [ ]:
pow(StrNum(base), StrNum(exponent), StrNum(modulus))